## Part2 Kaggle Competition

### Pre-poccessing
First,import some package that will be used.

In [1]:
import pandas as pd
import numpy as np
import json
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import keras

Read the json file and then normalize it to get the 'text','id' and rename its column name. Then,read the 'emotion','identification' file.

In [ ]:
df=pd.read_json("/home/neaf2070/code/ccy/dmlab2/DMlab2/DM2022-Lab2-Master-main/kaggle/tweets_DM.json",lines=True,orient='columns')
op=pd.json_normalize(df._source)
op=op.rename(columns={'tweet.tweet_id':'tweet_id','tweet.text':'text'})
ident=pd.read_csv("/home/neaf2070/code/ccy/dmlab2/DMlab2/DM2022-Lab2-Master-main/kaggle/data_identification.csv")
emo=pd.read_csv("/home/neaf2070/code/ccy/dmlab2/DMlab2/DM2022-Lab2-Master-main/kaggle/emotion.csv")

And merge the files according to tweet_id and split them to get the train and test data .And drop the the hashtags column which is not important to predict emtion.

In [ ]:
total=pd.merge(op,ident,on='tweet_id')
total=total.drop(columns='tweet.hashtags')
train_data=total[total['identification']=='train']
test_data=total[total['identification']=='test']

Merge the emotion with the train data as labels. 

In [ ]:
train_data=pd.merge(train_data,emo,on='tweet_id')

Drop the identification column in train and test data.

In [ ]:
train_data=train_data.drop(columns='identification')
test_data=test_data.drop(columns='identification')

Finally,save the train and test data which are pre-processing .Pickle file can save the read time and space.

In [ ]:
train_data.to_pickle("train_df.pkl") 
test_data.to_pickle("test_df.pkl")

Read the train and test pickle files.

In [4]:
train_data=pd.read_pickle("D:\gitresp\dmlab2\DM2022-Lab2-Master-main\DM2022-Lab2-Master-main\\new\\train_df.pkl")
test_data=pd.read_pickle("D:\gitresp\dmlab2\DM2022-Lab2-Master-main\DM2022-Lab2-Master-main\\new\\test_df.pkl")

Print them to oberserve.

In [5]:
print(train_data.head())

   tweet_id                                               text       emotion
0  0x376b20  People who post "add me on #Snapchat" must be ...  anticipation
1  0x2d5350  @brianklaas As we see, Trump is dangerous to #...       sadness
2  0x1cd5b0                Now ISSA is stalking Tasha 😂😂😂 <LH>          fear
3  0x1d755c  @RISKshow @TheKevinAllison Thx for the BEST TI...           joy
4  0x2c91a8       Still waiting on those supplies Liscus. <LH>  anticipation


I found that '#' and "@' are not important to predict so i replace both of them .

In [6]:
train_data['text']=train_data['text'].replace('#',"")
train_data['text']=train_data['text'].replace('@',"")

Using bag of words to get the max_features appears with nltk.word_tokenize and fit it.

In [7]:
BOW = CountVectorizer(max_features=10000, tokenizer=nltk.word_tokenize)
BOW.fit(train_data['text'])

c:\Users\ccy\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\feature_extraction\text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


CountVectorizer(max_features=10000,
                tokenizer=<function word_tokenize at 0x00000242B8495040>)

Split train data with train and valid part.(85:15)

In [8]:
train_da, val_da = train_test_split(train_data, random_state=777, train_size=0.85)

Check the valid data

In [9]:
val_da

,tweet_id,text,emotion
320379,0x2968c1,Love isn’t all we need—love is all there is. <...,joy
526228,0x38798b,Happy @DoctorWho_BBCA day 💙 #WhovianLove <LH> ...,joy
916717,0x2320b6,I AM THE LUCKIEST PERSON! NO! JOKE! I AM REALL...,joy
728692,0x30e9da,Whooooa is Lord Sugar even Jewish?! This is 11...,sadness
1116015,0x32e844,"@cbs your app has issues, love the show <LH> ...",anger
...,...,...,...
941900,0x2ce6d9,@PositiveHitsPER 10K Reasons <LH>,joy
1251283,0x264ca3,@FBC_Shelby_NC @CleveCoSchools Wow! To God be ...,trust
1444155,0x34b044,<LH> bought crackers asked market b4 returning...,disgust
1260967,0x3125dc,@espn Everybody Sports Programing New York. #y...,disgust


Change the variable name.

In [10]:
tr_x=train_da
val_x=val_da

Transform them to train and valid data.And set the train_y which is called labels.

In [11]:
train_x=BOW.transform(tr_x['text'])
val_x=BOW.transform(val_x['text'])
train_y=tr_x['emotion']


Also set the variable with labels of valid data.

In [12]:
val_y=val_da['emotion']

Transform test data with bag of words.

In [13]:
test_x=BOW.transform(test_data['text'])

### Model

#### 1. Logistic regression model
First, i test the Logisticregression model and the valid accuracy up to 0.5427.

In [14]:
from sklearn.linear_model import LogisticRegression
lg_model=LogisticRegression(max_iter=200,verbose=True,C=0.5)
lg_model=lg_model.fit(train_x,train_y)
val_pre_lg=lg_model.predict(val_x)
train_pre_lg=lg_model.predict(train_x)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
c:\Users\ccy\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.7min finished


And the train and test accuracy are similar .Maybe it is not train well so i raise the epochs but the valid accuracy is lower the the origin. Finally i choose the origin set to predict.

In [15]:
from sklearn.metrics import accuracy_score
print("train accu:",accuracy_score(train_pre_lg,train_y))
print("val accu:",accuracy_score(val_pre_lg,val_y))

train accu: 0.5487783981610503
val accu: 0.5427302081663499


In [16]:
test_pre_lg=lg_model.predict(test_x)

Save the test pedict result.

In [17]:
output= pd.DataFrame({'id':test_data.tweet_id , 'emotion': test_pre_lg})
output.to_csv('lg_pred.csv', index=False)

Change the labels types with binary using encoder ,decoder.Use this way can let the string labels to binary value and it can train on the linear model and MLP regression model.

In [18]:
from sklearn.preprocessing import LabelEncoder
import keras
label_encoder = LabelEncoder()
label_encoder.fit(train_y)
print( label_encoder.classes_)


def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

y_train = label_encode(label_encoder, train_y)

['anger' 'anticipation' 'disgust' 'fear' 'joy' 'sadness' 'surprise'
 'trust']


#### 2.Linear model
I test the linear model find that the valid accuracy is 0.539

In [19]:
from sklearn import linear_model
li_model=linear_model.LinearRegression()
li_model=li_model.fit(train_x,y_train)
li_pre=li_model.predict(train_x)
li_pre = label_decode(label_encoder, li_pre)
li_pre_val=li_model.predict(val_x)
li_pre_val = label_decode(label_encoder, li_pre_val)


In [20]:
from sklearn.metrics import accuracy_score
print("train accu:",accuracy_score(li_pre,train_y))
print("val accu:",accuracy_score(li_pre_val,val_y))

train accu: 0.5469323358346239
val accu: 0.5395012251814871


Train the test data which are our kaggle compete.

In [21]:
li_pre_te=li_model.predict(test_x)
li_pre_te= label_decode(label_encoder, li_pre_te)

Save the predict result.

In [22]:
output3= pd.DataFrame({'id':test_data.tweet_id , 'emotion': li_pre_te})
output3.to_csv('li_pre.csv', index=False)

#### 3.MLP regressor model

I try the mlp regressor model and set the 3 hidden layers each with 32 activations.We can find that the valid accuracy up to 0.573. 

In [34]:
from sklearn.neural_network import MLPRegressor
NN=MLPRegressor(activation='logistic',solver='adam',alpha=0.00007,max_iter=10,hidden_layer_sizes=(32,32,32),verbose=True,early_stopping=False)
NN=NN.fit(train_x,y_train)


Iteration 1, loss = 0.04124775
Iteration 2, loss = 0.03878980
Iteration 3, loss = 0.03847716
Iteration 4, loss = 0.03822937
Iteration 5, loss = 0.03796708
Iteration 6, loss = 0.03772373
Iteration 7, loss = 0.03755805
Iteration 8, loss = 0.03745556
Iteration 9, loss = 0.03740067
Iteration 10, loss = 0.03737777


c:\Users\ccy\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(


In [35]:
from sklearn.metrics import accuracy_score
mlp_pre=NN.predict(train_x)
mlp_pre_val=NN.predict(val_x)
mlp_pre=label_decode(label_encoder,mlp_pre)
mlp_pre_val=label_decode(label_encoder,mlp_pre_val)
print("train accu:",accuracy_score(mlp_pre,train_y))
print("val accu:",accuracy_score(mlp_pre_val,val_y))

train accu: 0.602415237935126
val accu: 0.5731101289303135


In [30]:
mlp_pre_test=NN.predict(test_x)
mlp_pre_test=label_decode(label_encoder,mlp_pre_test)
output5= pd.DataFrame({'id':test_data.tweet_id , 'emotion': mlp_pre_test})
output5.to_csv('mlp_pre_final.csv', index=False)

#### 4.MLP classifier model

Using the three hidden layers and each with 32 activation we can find the valid accuracy up to 0.566.

In [36]:
from sklearn.neural_network import MLPClassifier
NN2=MLPClassifier(activation='tanh',solver='adam',alpha=0.001,power_t=0.5,max_iter=15,hidden_layer_sizes=(32,32,32),verbose=True,early_stopping=False)
NN2=NN2.fit(train_x,train_y)

Iteration 1, loss = 1.28249268
Iteration 2, loss = 1.20034489
Iteration 3, loss = 1.17032966
Iteration 4, loss = 1.15259740
Iteration 5, loss = 1.14072918
Iteration 6, loss = 1.13118169
Iteration 7, loss = 1.12398284
Iteration 8, loss = 1.11776398
Iteration 9, loss = 1.11273163
Iteration 10, loss = 1.10823756
Iteration 11, loss = 1.10443726
Iteration 12, loss = 1.10076523
Iteration 13, loss = 1.09773197
Iteration 14, loss = 1.09468995
Iteration 15, loss = 1.09202421


c:\Users\ccy\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15) reached and the optimization hasn't converged yet.
  warnings.warn(


In [32]:
mlp_pre2=NN2.predict(train_x)
mlp_pre_val2=NN2.predict(val_x)
print("train accu:",accuracy_score(mlp_pre2,train_y))
print("val accu:",accuracy_score(mlp_pre_val2,val_y))

train accu: 0.636182659946267
val accu: 0.5662353722490668


In [33]:
mlp_pre_test2=NN2.predict(test_x)
output5= pd.DataFrame({'id':test_data.tweet_id , 'emotion': mlp_pre_test2})
output5.to_csv('mlp_pre_cla.csv', index=False)

Finally,i choose the MLP regressor to be my predict model because it has the highest accuracy than other models.

### The things i find
If choose the more bag of words to input to model .The accuracy will raise.The activation in hidden layer should be identity,the efficient is higher the use other different activations.The more activations MLP model has the higher accuracy the model can predict.But the time will raise simutaneously.  